# NDF-RT parser
Tenzen Rabgang and Romana Pernisch

In [ ]:
import xmltodict
import os
from owlready2 import *

In [3]:
file_name_xml = './data_ndfrt/NDFRT_Public_2018.02.05_TDE_inferred.xml'

xml_dict = None
with open(file_name_xml, 'r') as xml:
    xml_dict = xmltodict.parse(xml.read())

In [4]:
# owl:Ontology
namespace = xml_dict['terminology']['namespaceDef']

# owl:ObjectProperty
roles = xml_dict['terminology']['roleDef']
associations = xml_dict['terminology']['associationDef']

# owl:DatatypeProperty
# owl:AnnotationProperty???
properties = xml_dict['terminology']['propertyDef']

# owl:Class
# concepts = xml_dict['terminology']['conceptDef']
concepts = xml_dict['terminology']['conceptInf']

# merged to ObjectProperty
kinds = xml_dict['terminology']['kindDef']

# ???
qualifiers = xml_dict['terminology']['qualifierDef']

In [5]:
headConcepts = []
for concept in concepts:
    if concept['definingConcepts'] is None:
        headConcepts.append(concept)

In [6]:
# (unique identifier of NDF-RT)
NUI_Code = ''
for props in properties:
    if props['name'] == "NUI":
        NUI_Code = props['code']

In [7]:
code_kinds_map = {}
for kind in kinds:
    code_kinds_map[kind['code']] = kind

In [8]:
kind_head_code_map = {}
for head in headConcepts:
    if head['kind'] in code_kinds_map:
        kind_head_code_map[head['kind']] = head['code']

In [9]:
code_property_map = {}
code_roles_map = {}

for props in properties:
    code_property_map[props['code']] = props
for __role in roles:
    code_roles_map[__role['code']] = __role

In [10]:
def create_owl_class(onto, concept, owl_dict, owl_roles):
    with onto:
        value = ""
        for props in concept['properties']['property']:
            if props['name'] == NUI_Code:
                value = props['value']

        onto_class = types.new_class(value, (Thing,))
        onto_class.label = concept['name']
        onto_class.code = concept['code']

        # creating annotations/tags
        if concept['properties']:
            for props in concept['properties']['property']:
                code = props['name']
                if code in code_property_map:
                    props_name = code_property_map[code]['name']
                    annot_props = types.new_class(props_name, (AnnotationProperty,))
                    setattr(onto_class, props_name, props['value'])
        
        owl_dict[concept['code']] = onto_class
    
    return (owl_dict, owl_roles)

In [11]:
onto = get_ontology("test5")

In [12]:
# create data properties
owlProps = {}
for props in properties:
    with onto:
        code = types.new_class('code', (AnnotationProperty,))
        
        onto_data_property = types.new_class(props['name'],  (DataProperty,))
        onto_data_property.label = props['name']
        onto_data_property.code = props['code']
        
        owlProps[props['name']] = onto_data_property

In [13]:
# create object properties
owlRoles = {}
for role in roles:
    with onto:
        onto_data_property = types.new_class(role['code'],  (ObjectProperty,))
        onto_data_property.code = role['code']
        onto_data_property.label = role['name']

        owlRoles[role['code']] = onto_data_property

In [14]:
# create top-most classes
owlClasses = {}
for concept in headConcepts:
    owlClasses, owlRoles = create_owl_class(onto, concept, owlClasses, owlRoles)

In [15]:
for concept in concepts:
    if concept['code'] in owlClasses:
        continue
    else:
        owlClasses, owlRoles = create_owl_class(onto, concept, owlClasses, owlRoles)

In [16]:
for concept in concepts:
    current_class = owlClasses[concept['code']]
    
    # add hierarchy
    if concept['definingConcepts']:
        if type(concept['definingConcepts']['concept']) == str:
            concept['definingConcepts']['concept'] = [concept['definingConcepts']['concept']]
        
        for conc in concept['definingConcepts']['concept']:
            current_class.is_a.append(owlClasses[conc])
    
    # add relations
    if concept['definingRoles']:
        _role = concept['definingRoles']['role']
        concept['definingRoles']['role'] = [_role] if not type(_role) == list else _role

        for role in concept['definingRoles']['role']:
            if role['name'] in code_roles_map and role['value'] in owlClasses:
                if role['name'] in owlRoles:
                    new_prop = owlRoles[role['name']]
                    current_class.is_a.append(new_prop.some(owlClasses[role['value']]))

In [17]:
# add domain/range to roles/ObjectProperty
for role in roles:
    with onto:
        role_props = owlRoles[role['code']]
        if role['domain'] in kind_head_code_map:
            code_str = kind_head_code_map[role['domain']]
            if code_str in owlClasses:
                role_props.domain = owlClasses[code_str]
        if role['range'] in kind_head_code_map:
            code_str = kind_head_code_map[role['range']]
            if code_str in owlClasses:
                role_props.range = owlClasses[code_str]

In [18]:
onto.save('test2.owl')